In [ ]:
## PART 5: EXTRACTING SDF AND PDBQT FILES 

In [ ]:
### select top 500 candidates from virtual_screening_results.csv 
## Then extract sdf from top-500-compounds then converted to pdbqt files using openbabel 
# saved in a folder for further molecular docking

In [ ]:
!pip install rdkit

In [ ]:
## install open babel
!apt-get install openbabel

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openbabel is already the newest version (3.1.1+dfsg-6ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import os

In [2]:
# Load VS-screened results file
df = pd.read_csv("virtual_screening_results.csv")
df.head()

,Common_Name,SMILES,MorganFingerprint,Predicted_Prob,Predicted_Activity
0,(3Z)-6-(4-HYDROXY-3-METHOXYPHENYL)-3-(1H-PYRRO...,COc1cc(-c2ccc3c(c2)NC(=O)/C3=C\c2ccc[nH]2)ccc1O,[0 0 0 ... 0 0 0],0.997185,1
1,PEN-866,CCc1c2c(nc3ccc(OC(=O)N4CCC(CCn5ccc6cc(-n7c(-c8...,[0 1 0 ... 0 0 0],0.994814,1
2,"Furo[2,3d]Pyrimidine Antifolate",CN(Cc1coc2nc(N)nc(N)c12)c1ccc(C(=O)N[C@@H](CCC...,[0 1 0 ... 0 0 0],0.994609,1
3,Carotegrast methyl,COC(=O)[C@H](Cc1ccc(-n2c(=O)c3cc(N(C)C)ccc3n(C...,[0 1 0 ... 0 0 0],0.991697,1
4,"4-{5-[(Z)-(2-Imino-4-Oxo-1,3-Thiazolidin-5-Yli...",N=C1NC(=O)/C(=C/c2ccc(-c3ccc(S(N)(=O)=O)cc3)o2)S1,[0 0 0 ... 0 0 0],0.991124,1


In [3]:
# Select top 500 based on Predicted Probability
top500 = df.sort_values(by='Predicted_Prob', ascending=False).head(500)
top500.head()

,Common_Name,SMILES,MorganFingerprint,Predicted_Prob,Predicted_Activity
0,(3Z)-6-(4-HYDROXY-3-METHOXYPHENYL)-3-(1H-PYRRO...,COc1cc(-c2ccc3c(c2)NC(=O)/C3=C\c2ccc[nH]2)ccc1O,[0 0 0 ... 0 0 0],0.997185,1
1,PEN-866,CCc1c2c(nc3ccc(OC(=O)N4CCC(CCn5ccc6cc(-n7c(-c8...,[0 1 0 ... 0 0 0],0.994814,1
2,"Furo[2,3d]Pyrimidine Antifolate",CN(Cc1coc2nc(N)nc(N)c12)c1ccc(C(=O)N[C@@H](CCC...,[0 1 0 ... 0 0 0],0.994609,1
3,Carotegrast methyl,COC(=O)[C@H](Cc1ccc(-n2c(=O)c3cc(N(C)C)ccc3n(C...,[0 1 0 ... 0 0 0],0.991697,1
4,"4-{5-[(Z)-(2-Imino-4-Oxo-1,3-Thiazolidin-5-Yli...",N=C1NC(=O)/C(=C/c2ccc(-c3ccc(S(N)(=O)=O)cc3)o2)S1,[0 0 0 ... 0 0 0],0.991124,1


In [4]:
# Save top 500 to CSV for docking
top500.to_csv("top500_for_docking.csv", index=False)

print("Top 500 compounds saved to top500_for_docking.csv")

Top 500 compounds saved to top500_for_docking.csv


In [ ]:
# Load CSV
data = pd.read_csv("/content/top_500_for_docking.csv")
data.head()

,Common_Name,SMILES,MorganFingerprint,Predicted_Prob,Predicted_Activity
0,(3Z)-6-(4-HYDROXY-3-METHOXYPHENYL)-3-(1H-PYRRO...,COc1cc(-c2ccc3c(c2)NC(=O)/C3=C\c2ccc[nH]2)ccc1O,[0 0 0 ... 0 0 0],0.997185,1
1,PEN-866,CCc1c2c(nc3ccc(OC(=O)N4CCC(CCn5ccc6cc(-n7c(-c8...,[0 1 0 ... 0 0 0],0.994814,1
2,"Furo[2,3d]Pyrimidine Antifolate",CN(Cc1coc2nc(N)nc(N)c12)c1ccc(C(=O)N[C@@H](CCC...,[0 1 0 ... 0 0 0],0.994609,1
3,Carotegrast methyl,COC(=O)[C@H](Cc1ccc(-n2c(=O)c3cc(N(C)C)ccc3n(C...,[0 1 0 ... 0 0 0],0.991697,1
4,"4-{5-[(Z)-(2-Imino-4-Oxo-1,3-Thiazolidin-5-Yli...",N=C1NC(=O)/C(=C/c2ccc(-c3ccc(S(N)(=O)=O)cc3)o2)S1,[0 0 0 ... 0 0 0],0.991124,1


In [ ]:
# Create folder to save SDFs
output_dir = "Ligand_SDFs"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Loop through each compound
for i, row in data.iterrows():
    name = row['Common_Name']
    smiles = row['SMILES']

    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Skipping {name}: Invalid SMILES")
        continue

    mol = Chem.AddHs(mol)

    # Embed molecule, check if it succeeded
    embed_status = AllChem.EmbedMolecule(mol, randomSeed=42)
    if embed_status != 0:
        print(f"Skipping {name}: Could not embed 3D coordinates")
        continue

    # Optimize geometry
    AllChem.UFFOptimizeMolecule(mol)

    # Create safe file name
    safe_name = "".join(c if c.isalnum() else "_" for c in name)
    sdf_path = os.path.join(output_dir, f"{i+1}_{safe_name}.sdf")

    # Write SDF
    writer = Chem.SDWriter(sdf_path)
    writer.write(mol)
    writer.close()

    print(f"Saved {sdf_path}")

Saved Ligand_SDFs/1__3Z__6__4_HYDROXY_3_METHOXYPHENYL__3__1H_PYRROL_2_YLMETHYLENE__1_3_DIHYDRO_2H_INDOL_2_ONE.sdf
Saved Ligand_SDFs/2_PEN_866.sdf
Saved Ligand_SDFs/3_Furo_2_3d_Pyrimidine_Antifolate.sdf
Saved Ligand_SDFs/4_Carotegrast_methyl.sdf
Saved Ligand_SDFs/5_4__5___Z___2_Imino_4_Oxo_1_3_Thiazolidin_5_Ylidene_methyl_furan_2_Yl_benzenesulfonamide.sdf
Saved Ligand_SDFs/6_Dirocaftor.sdf
Saved Ligand_SDFs/7_Cytidine_5__Monophosphate_5_N_Acetylneuraminic_Acid.sdf
Saved Ligand_SDFs/8_10_Propargyl_5_8_Dideazafolic_Acid.sdf
Saved Ligand_SDFs/9_E_6005.sdf
Saved Ligand_SDFs/10__E__3__5__5__4_CHLOROPHENYL_FURAN_2_YL_METHYLENE__4_OXO_2_THIOXOTHIAZOLIDIN_3_YL_PROPANOIC_ACID.sdf
Saved Ligand_SDFs/11_2__4__DIMETHYLAMINO_PHENYL__6_HYDROXY_3_METHYL_1_3_BENZOTHIAZOL_3_IUM.sdf
Saved Ligand_SDFs/12_4__5___Z___2_IMINO_4_OXO_1_3_THIAZOLIDIN_5_YLIDENE_METHYL_FURAN_2_YL__2__TRIFLUOROMETHYL_BENZENESULFONAMIDE.sdf
Saved Ligand_SDFs/13_4__5___Z___2_4_DIOXO_1_3_THIAZOLIDIN_5_YLIDENE_METHYL_FURAN_2_YL_BENZENE

[15:11:22] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[15:11:22] UFFTYPER: Warning: hybridization set to SP3 for atom 44


Saved Ligand_SDFs/36_Tin_protoporphyrin_IX.sdf
Saved Ligand_SDFs/37_Icariin.sdf
Saved Ligand_SDFs/38_LGD_2226.sdf
Saved Ligand_SDFs/39_6__3_4_DIHYDROXYBENZYL__3_ETHYL_1__2_4_6_TRICHLOROPHENYL__1H_PYRAZOLO_3_4_D_PYRIMIDIN_4_5H__ONE.sdf


[15:11:23] UFFTYPER: Unrecognized atom type: Fe5+2 (34)
[15:11:23] UFFTYPER: Unrecognized atom type: Fe5+2 (34)


Saved Ligand_SDFs/40_12_Phenylheme.sdf
Saved Ligand_SDFs/41_Dactolisib.sdf
Saved Ligand_SDFs/42_Metavert.sdf
Saved Ligand_SDFs/43_4__dimethylamino__N__7__hydroxyamino__7_oxoheptyl_benzamide.sdf
Saved Ligand_SDFs/44_Verubulin.sdf
Saved Ligand_SDFs/45_Balaglitazone.sdf
Saved Ligand_SDFs/46_Tallimustine.sdf
Saved Ligand_SDFs/47_GTX_758.sdf
Saved Ligand_SDFs/48_4_epi_vancosaminyl_derivative_of_vancomycin.sdf
Saved Ligand_SDFs/49_Oftasceine.sdf
Saved Ligand_SDFs/50_Ellagic_acid.sdf
Saved Ligand_SDFs/51_Dabigatran.sdf
Saved Ligand_SDFs/52_4___4__4_fluoro_3_methylphenyl__1_3_thiazol_2_yl_amino__2_hydroxybenzoic_acid.sdf
Saved Ligand_SDFs/53_AGN_201904.sdf
Saved Ligand_SDFs/54_Volixibat.sdf
Saved Ligand_SDFs/55_Ombitasvir.sdf
Saved Ligand_SDFs/56_Rebastinib.sdf
Saved Ligand_SDFs/57_Dazucorilant.sdf
Saved Ligand_SDFs/58_Tasquinimod.sdf
Saved Ligand_SDFs/59_Trichostatin_A.sdf
Saved Ligand_SDFs/60_Zeranol.sdf
Saved Ligand_SDFs/61_2__4__2___3__5__PYRIDIN_2_YLTHIO_THIAZOL_2_YL_UREIDO_METHYL__1H_IMI

...

Skipping Cobamamide: Could not embed 3D coordinates
Saved Ligand_SDFs/348_Zibotentan.sdf
Saved Ligand_SDFs/349__3Z__1___6_fluoro_4H_1_3_benzodioxin_8_yl_methyl__4___E__2_phenylethenyl__1H_indole_2_3_dione_3_oxime.sdf
Saved Ligand_SDFs/350_BF_844.sdf
Saved Ligand_SDFs/351_5__3__3__2_4_DICHLORO_BENZOYL__UREIDO__2_METHYL_PHENOXY__PENTANOIC_ACID.sdf
Saved Ligand_SDFs/352_N__2__1_maleimidyl_ethyl__7_diethylaminocoumarin_3_carboxamide.sdf
Saved Ligand_SDFs/353_Vancomycin.sdf
Saved Ligand_SDFs/354_Eribaxaban.sdf
Saved Ligand_SDFs/355_3__5__DIBROMO_2__4_4__6__TETRAHYDROXY_AURONE.sdf
Saved Ligand_SDFs/356_LY231514_tetra_glu.sdf
Saved Ligand_SDFs/357_PASBN.sdf
Saved Ligand_SDFs/358_Crenolanib.sdf
Saved Ligand_SDFs/359_S_55746.sdf
Saved Ligand_SDFs/360_Mycophenolic_acid.sdf
Saved Ligand_SDFs/361_Balsalazide.sdf
Saved Ligand_SDFs/362_3__5_amino_7_hydroxy__1_2_3_triazolo_4_5_d_pyrimidin_2_yl__N__3_5_dichlorobenzyl__benzamide.sdf
Saved Ligand_SDFs/363_N__TRANS_4__NITRO_4_STILBENYL__N_METHYL_5_AMINO_

[15:14:06] UFFTYPER: Unrecognized atom type: Fe1+2 (0)
[15:14:06] UFFTYPER: Unrecognized atom type: Fe1+2 (42)


Saved Ligand_SDFs/383_2_Formyl_Protoporphryn_Ix.sdf
Saved Ligand_SDFs/384_N___5_CHLORO_1_3_BENZODIOXOL_4_YL__N__3_4_5__TRIMETHOXYPHENYL_PYRIMIDINE_2_4_DIAMINE.sdf
Saved Ligand_SDFs/385_Firsocostat.sdf
Saved Ligand_SDFs/386_Pemetrexed.sdf
Saved Ligand_SDFs/387__E__3_4_DIHYDROXY_N____2_METHOXYNAPHTHALEN_1_YL_METHYLENE_BENZOHYDRAZIDE.sdf
Saved Ligand_SDFs/388_Lumicitabine.sdf
Saved Ligand_SDFs/389_4__1_3_Benzodioxol_5_Yl__5__5_Ethyl_2_4_Dihydroxyphenyl__2h_Pyrazole_3_Carboxylic_Acid.sdf
Saved Ligand_SDFs/390_Barusiban.sdf
Saved Ligand_SDFs/391_Naphthalene_Trisulfonate.sdf
Saved Ligand_SDFs/392_BLZ_945.sdf
Saved Ligand_SDFs/393_tert_butyl___2_amino_4_oxo_4_7_dihydro_3H_pyrrolo_2_3_d_pyrimidin_5_yl_methyl_carbamate.sdf
Saved Ligand_SDFs/394_Carba_nicotinamide_adenine_dinucleotide.sdf
Saved Ligand_SDFs/395_4_amino_N__4__benzyloxy_phenyl_butanamide.sdf
Saved Ligand_SDFs/396_Risdiplam.sdf
Saved Ligand_SDFs/397_3___4___5_CHLORO_1_3_BENZODIOXOL_4_YL_AMINO_PYRIMIDIN_2_YL_AMINO_BENZAMIDE.sdf
Saved

[15:14:13] UFFTYPER: Unrecognized charge state for atom: 1
[15:14:13] UFFTYPER: Unrecognized charge state for atom: 1


Saved Ligand_SDFs/445_3__4_Phenylamino_Phenylamino__2__1h_Tetrazol_5_Yl__Acrylonitrile.sdf
Saved Ligand_SDFs/446_Nifurpirinol.sdf
Saved Ligand_SDFs/447_Diosmetin.sdf
Saved Ligand_SDFs/448_Bucetin.sdf
Saved Ligand_SDFs/449_Lomibuvir.sdf
Saved Ligand_SDFs/450__4__4_ACETYLAMINO_PHENYL__3_5_DIOXO_4_AZA_TRICYCLO_5_2_2_0_2_6_UNDEC_1_YLCARBAMOYLOXY__ACETIC_ACID.sdf
Saved Ligand_SDFs/451_Edivoxetine.sdf
Saved Ligand_SDFs/452_3__9_HYDROXY_1_3_DIOXO_4_PHENYL_2_3_DIHYDROPYRROLO_3_4_C_CARBAZOL_6_1H__YL_PROPANOIC_ACID.sdf
Saved Ligand_SDFs/453_N__4_sulfamoylphenyl__1H_indazole_3_carboxamide.sdf
Saved Ligand_SDFs/454_Rabeximod.sdf
Saved Ligand_SDFs/455_GW810781.sdf
Saved Ligand_SDFs/456_KB_141.sdf
Saved Ligand_SDFs/457_Tuspetinib.sdf
Saved Ligand_SDFs/458_Aclarubicin.sdf
Saved Ligand_SDFs/459_CM_4307.sdf
Saved Ligand_SDFs/460_VK_0214.sdf
Saved Ligand_SDFs/461_N__4__benzyloxy_phenyl_glycinamide.sdf
Saved Ligand_SDFs/462_SPG302.sdf
Saved Ligand_SDFs/463_4___7_OXO_7H_THIAZOLO_5_4_E_INDOL_8_YLMETHYL__AM

In [ ]:
# the ligands that were skipped because 3D coordinates could not be embedded are:

# Ukrain cation

# Nerofe

# 2-Acetyl-Protoporphyrin Ix

# Oritavancin

# Cobamamide

# So, 5 ligands in total were skipped.

In [ ]:
!ls Ligand_SDFs | wc -l

494


In [ ]:
# Convert SDF → PDBQT

# Use OpenBabel or AutoDock Tools (ADT) to convert SDF to PDBQT for Vina docking.

# Example using OpenBabel:

import os

# Paths
input_folder = "Ligand_SDFs"
output_folder = "Ligand_PDBQT"
os.makedirs(output_folder, exist_ok=True)

# Loop through all SDF files
for f in os.listdir(input_folder):
    if f.endswith(".sdf"):
        input_file = os.path.join(input_folder, f)
        base = os.path.splitext(f)[0]
        output_file = os.path.join(output_folder, base + ".pdbqt")
        # Command using OpenBabel
        os.system(f"obabel {input_file} -O {output_file} --gen3d")




In [ ]:
import os

folder = "Ligand_PDBQT"   # or "Ligand_SDFs"
files = [f for f in os.listdir(folder) if f.endswith(".pdbqt")]  # change extension if needed
print(f"Number of PDBQT files: {len(files)}")


Number of PDBQT files: 212


In [ ]:
print("First 10 files:", files[:10])


First 10 files: ['458_Aclarubicin.pdbqt', '144_Chrysin.pdbqt', '214_MK_4965.pdbqt', '348_Zibotentan.pdbqt', '468_N__P_CYANOPHENYL__N__DIPHENYLMETHYL_GUANIDINE_ACETIC_ACID.pdbqt', '187_GSK_1292263.pdbqt', '421_Genistein.pdbqt', '449_Lomibuvir.pdbqt', '398_Ro_12_7310.pdbqt', '401_Omeprazole.pdbqt']


In [ ]:
import os

input_folder = "Ligand_SDFs"
output_folder = "Ligand_PDBQT"
os.makedirs(output_folder, exist_ok=True)

# Get list of already converted ligands
converted = {os.path.splitext(f)[0] for f in os.listdir(output_folder) if f.endswith(".pdbqt")}

sdf_files = [f for f in os.listdir(input_folder) if f.endswith(".sdf")]
print(f"Total SDF files: {len(sdf_files)}")
print(f"Already converted: {len(converted)}")

# Resume conversion
remaining = 0
for f in sdf_files:
    base = os.path.splitext(f)[0]
    if base in converted:
        continue  # skip already converted
    input_file = os.path.join(input_folder, f)
    output_file = os.path.join(output_folder, base + ".pdbqt")
    os.system(f"obabel {input_file} -O {output_file} --gen3d")
    remaining += 1
    print(f"Converted: {base}")

print(f"✅ Resumed and converted {remaining} new ligands.")


Total SDF files: 494
Already converted: 494
✅ Resumed and converted 0 new ligands.


In [ ]:
import os

output_folder = "Ligand_PDBQT"
num_files = len([f for f in os.listdir(output_folder) if f.endswith(".pdbqt")])
print(f"Number of PDBQT files: {num_files}")


Number of PDBQT files: 494


In [ ]:
## Download and zip

!zip -r Ligand_SDFs.zip Ligand_SDFs
!zip -r Ligand_PDBQT.zip Ligand_PDBQT


updating: Ligand_PDBQT/ (stored 0%)
updating: Ligand_PDBQT/458_Aclarubicin.pdbqt (deflated 78%)
updating: Ligand_PDBQT/144_Chrysin.pdbqt (deflated 74%)
updating: Ligand_PDBQT/214_MK_4965.pdbqt (deflated 74%)
updating: Ligand_PDBQT/348_Zibotentan.pdbqt (deflated 75%)
updating: Ligand_PDBQT/468_N__P_CYANOPHENYL__N__DIPHENYLMETHYL_GUANIDINE_ACETIC_ACID.pdbqt (deflated 75%)
updating: Ligand_PDBQT/187_GSK_1292263.pdbqt (deflated 75%)
updating: Ligand_PDBQT/421_Genistein.pdbqt (deflated 74%)
updating: Ligand_PDBQT/449_Lomibuvir.pdbqt (deflated 76%)
updating: Ligand_PDBQT/398_Ro_12_7310.pdbqt (deflated 75%)
updating: Ligand_PDBQT/401_Omeprazole.pdbqt (deflated 74%)
updating: Ligand_PDBQT/269_Chiauranib.pdbqt (deflated 75%)
updating: Ligand_PDBQT/76_Quercetin_3__O_phosphate.pdbqt (deflated 76%)
updating: Ligand_PDBQT/153_N___5_chloro_1_3_benzodioxol_4_yl__N__3_methylsulfonylphenyl_pyrimidine_2_4_diamine.pdbqt (deflated 74%)
updating: Ligand_PDBQT/169_Ciluprevir.pdbqt (deflated 77%)
updating: L

In [ ]:
from google.colab import files

files.download("Ligand_SDFs.zip")
files.download("Ligand_PDBQT.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>